In [ ]:
!pip install segmentation_models_pytorch
!pip install wandb
!pip install torcheval

!pip install torchmetrics

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Iniatial

In [ ]:
!mkdir Dataset
!mkdir Dataset/COMOFOD

!wget -P Dataset/COMOFOD https://www.vcl.fer.hr/comofod/comofod_small.rar

--2024-05-22 10:13:24--  https://www.vcl.fer.hr/comofod/comofod_small.rar
Resolving www.vcl.fer.hr (www.vcl.fer.hr)... 161.53.16.144
Connecting to www.vcl.fer.hr (www.vcl.fer.hr)|161.53.16.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3213318957 (3.0G) [application/octet-stream]
Saving to: ‘Dataset/COMOFOD/comofod_small.rar’

comofod_small.rar   100%[===================>]   2.99G  1.80MB/s    in 27m 37s 

2024-05-22 10:41:01 (1.85 MB/s) - ‘Dataset/COMOFOD/comofod_small.rar’ saved [3213318957/3213318957]



In [3]:
!unzip /content/drive/MyDrive/Dataset.zip -d /content

Archive:  /content/drive/MyDrive/Dataset.zip
 extracting: /content/findit/val_json.txt  
 extracting: /content/findit/train_json.txt  
 extracting: /content/findit/test_json.txt  
 extracting: /content/findit/findit2/val.txt  
 extracting: /content/findit/findit2/test.txt  
 extracting: /content/findit/findit2/train.txt  
 extracting: /content/findit/findit2/val_bm/X51006828201.png  
 extracting: /content/findit/findit2/val_bm/X51008114283.png  
 extracting: /content/findit/findit2/val_bm/X51005441401.png  
 extracting: /content/findit/findit2/val_bm/X51005757304.png  
 extracting: /content/findit/findit2/val_bm/X51005605333.png  
 extracting: /content/findit/findit2/val_bm/X51005711404.png  
 extracting: /content/findit/findit2/val_bm/X00016469612.png  
 extracting: /content/findit/findit2/val_bm/X51005568890.png  
 extracting: /content/findit/findit2/val_bm/X51007419197.png  
 extracting: /content/findit/findit2/val_bm/X51006414636.png  
 extracting: /content/findit/findit2/val_bm/X5

In [ ]:
!rm -r /content/Dataset/COMOFOD/comofod_small.rar

In [ ]:
# import shutil

# shutil.rmtree('/Project/Dataset', ignore_errors=True)

# Libraries

In [1]:
# For managing COCO dataset
# from pycocotools.coco import COCO

# For creating and managing folder/ files
import glob
import os
import shutil

# For managing images
from PIL import Image
import skimage.io as io

# Basic libraries
import numpy as np
import pandas as pd
import random
import cv2

# For plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import wandb

# For importing models and working with them
## Torch
import torch
import torch.utils.data # for Dataset
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp

## Torchvision
import torchvision
from torchvision.transforms import transforms

# For creating train - test splits
from sklearn.model_selection import train_test_split

import pathlib
import pylab
import requests
from io import BytesIO
from pprint import pprint
from tqdm import tqdm
import time
from imutils import paths

# Performance Metrics
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix


# Functions - to have them separated in files
import CalcMetrics as cm
import BinaryMasks as bm
import TrainModel as trModel
import ValidateModel as valModel
import TestModel as testModel
import CreateDataset_Comofod as com
import CreateDataset_IMD2020 as imd
import CreateDataset_SROIE as sroie
import DisplayMetrics as dm
import PlotResults as pr
import MainLoop as main
import EarlyStopping as stopping


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# %matplotlib inline

cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda


# Data

## Data Preparation

In [2]:
RESNET_SIZE = 256
transf_size = RESNET_SIZE

# Initialize transformations - train
transforms_train = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize transformations - test (optional)
transforms_test = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

BATCH_SIZE = 8

In [9]:
# Train
csv_file = '/content/train.csv'
images_dir = '/content/findit/findit2/train'
binary_masks_dir = '/content/findit/findit2/train_bm'
bounding_box_masks_dir = '/content/findit/findit2/train_masks'

# Create dataset and data loader
dataset = sroie.SegmentationDataset(csv_file, images_dir, binary_masks_dir, bounding_box_masks_dir, transform=transforms_train)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
# Test
csv_file = '/content/test.csv'
images_dir = '/content/findit/findit2/test'
binary_masks_dir = '/content/findit/findit2/test_bm'
bounding_box_masks_dir = '/content/findit/findit2/test_masks'

# Create dataset and data loader
dataset = sroie.SegmentationDataset(csv_file, images_dir, binary_masks_dir, bounding_box_masks_dir, transform=transforms_test)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
# Val
csv_file = '/content/val.csv'
images_dir = '/content/findit/findit2/val'
binary_masks_dir = '/content/findit/findit2/val_bm'
bounding_box_masks_dir = '/content/findit/findit2/val_masks'

# Create dataset and data loader
dataset = sroie.SegmentationDataset(csv_file, images_dir, binary_masks_dir, bounding_box_masks_dir, transform=transforms_test)
val_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
# def display_images(dataset, num_images=9):
#     total_images = min(num_images, len(dataset))
#     num_cols = 3  # Number of columns in the visualization grid
#     num_rows = (total_images + num_cols - 1) // num_cols  # Calculate number of rows

#     fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))

#     for i, ax in enumerate(axes.flat):
#         if i < total_images:
#             original_img, bounding_box_mask, binary_mask = dataset[i]

#             # Original image
#             ax.imshow(original_img.permute(1, 2, 0))  # Assuming original_img is in PyTorch tensor format
#             ax.set_title('Original Image')
#             ax.axis('off')

#             # Bounding box mask
#             ax = axes.flat[i + num_cols]
#             ax.imshow(bounding_box_mask.permute(1, 2, 0))  # Assuming bounding_box_mask is in PyTorch tensor format
#             ax.set_title('Bounding Box Mask')
#             ax.axis('off')

#             # Binary mask
#             ax = axes.flat[i + 2 * num_cols]
#             ax.imshow(binary_mask.squeeze(), cmap='gray')  # Assuming binary_mask is in PyTorch tensor format
#             ax.set_title('Binary Mask')
#             ax.axis('off')
#         else:
#             ax.axis('off')

#     plt.tight_layout()
#     plt.show()

# # Assuming you have initialized your dataset object
# # dataset = SegmentationDataset(csv_file, images_dir, binary_masks_dir, bounding_box_masks_dir, transform=transforms.ToTensor())

# # Display images from the dataset
# display_images(dataset)


In [6]:
# # Train
# csv_file = '/content/train.csv'
# images_dir = '/content/findit/findit2/train'
# binary_masks_dir = '/content/findit/findit2/train_bm'
# bounding_box_masks_dir = '/content/findit/findit2/train_masks'

# train_original_images, train_bounding_box_masks, train_binary_masks  = sroie.load_sroie_dataset(csv_file, images_dir, binary_masks_dir, bounding_box_masks_dir)

In [7]:
# # Test
# csv_file = '/content/test.csv'
# images_dir = '/content/findit/findit2/test'
# binary_masks_dir = '/content/findit/findit2/test_bm'
# bounding_box_masks_dir = '/content/findit/findit2/test_masks'

# test_original_images, test_bounding_box_masks, test_binary_masks  = sroie.load_sroie_dataset(csv_file, images_dir, binary_masks_dir, bounding_box_masks_dir)

In [8]:
# # Val
# csv_file = '/content/val.csv'
# images_dir = '/content/findit/findit2/val'
# binary_masks_dir = '/content/findit/findit2/val_bm'
# bounding_box_masks_dir = '/content/findit/findit2/val_masks'

# val_original_images, val_bounding_box_masks, val_binary_masks  = sroie.load_sroie_dataset(csv_file, images_dir, binary_masks_dir, bounding_box_masks_dir)

In [9]:
# len(train_original_images)

In [10]:
# train_original_images[1]

In [11]:
# train_bounding_box_masks[1]

In [12]:
# train_binary_masks[1]

In [13]:
# original_images = [train_original_images, test_original_images, val_original_images]
# altered_images = [train_bounding_box_masks, test_bounding_box_masks, val_bounding_box_masks]
# masks = [train_binary_masks, test_binary_masks, val_binary_masks]


original_images = train_loader
altered_images = test_loader
masks = val_loader

AttributeError: 'DataLoader' object has no attribute 'images'

## Main

In [ ]:
# 73c482950d37419655b401f882c34c719ca587af

RESNET_SIZE = 256
transf_size = RESNET_SIZE

# Initialize transformations - train
transforms_train = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize transformations - test (optional)
transforms_test = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize the U-Net model
# unet = smp.Unet(
#                 encoder_name = "resnet101",
#                 encoder_weights = "imagenet",
#                 in_channels = 3,  # 3 channels for the image
#                 classes = 1,  # 1 class => binary mask
#                 activation = 'sigmoid'
#                ).to(device)


# Define hyperparameters
INIT_LR = 0.0001
BATCH_SIZE = 8
NUM_EPOCHS = [10, 20, 50, 100, 150]
TEST_SPLIT = [0.1, 0.2] # 0.3
VALID_SPLIT = 0.2 # 0.1

model_type = 'GCA' # unet
channels = 3 # 6
dataset_type = 'sroie' # imd, sroie, comofod

wb_name = "GCANet_SROIE_imgs"

main.main_loop(original_images, altered_images, masks, transforms_train, transforms_test,
              model_type, channels, dataset_type,
              wb_name, INIT_LR, BATCH_SIZE, NUM_EPOCHS, TEST_SPLIT, VALID_SPLIT)



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[INFO] TEST_SPLIT = 0.1 ...
Splits, Datasets, and Dataloaders
[INFO] Total time taken to create the dataset and dataloader: 0.00s
trainSteps = 72, testSteps = 27, valSteps = 24


[INFO] Training the network for 10 epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Training...
Validating...


 10%|█         | 1/10 [02:18<20:44, 138.23s/it]

Training...
Validating...


 20%|██        | 2/10 [04:37<18:29, 138.65s/it]

Training...


TypeError: plot_results() takes 1 positional argument but 5 were given

In [3]:
a = [1, 1]
b = [2, 2]
c = 3

d = [a, b, c]
d[0]

[1, 1]